### General idea
1. Clean dataset => dfClean
2. Vectorize words => to probability density
3. Perform logistic regression on vectorized words 
    of scales of reviews 0 (0,.1),1 (.2,.3) ,2 (.4,.5),3 (.6,.7) ,4 (.8,.9, 1) reviews

In [1]:
# Read data set and stop words
import pandas as pd
import re 
import nltk
#nltk.download('stopwords')
df1 = pd.read_csv (r'.\AppReview.csv')
df=df1
len(df.index)


111143

In [2]:
# Corpus of stop words
from nltk.corpus import stopwords

In [3]:
# processes a review and returns a list of words
def review_to_words(review, string = True, remove_stopwords=True):
    # Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    review_text=review
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # Convert words to lower case and split them
    words = review_text.lower().split()
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    if string:
        return " ".join(words)
    else:
        return words

In [4]:
#Clean up text
#Remove non-ascii text
#Remove all rows missing reviewerName
def fixString(x):
    return x.encode('ascii',errors='ignore')

# df[["reviewText"]]=df[["reviewText"]].apply(lambda x: str(x["reviewText"]).encode('ascii',errors='ignore').decode(), axis=1)
# df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)
df[["reviewText"]]=df[["reviewText"]].apply(lambda x: review_to_words(x["reviewText"]), axis=1)
df[["reviewerName"]]=df[["reviewerName"]].apply(lambda x: str(x["reviewerName"]).encode('ascii',errors='ignore').decode(), axis=1)

dfCleaned=df[df['reviewText'].str.strip().astype(bool)]
dfCleaned=dfCleaned[df['reviewerName'].str.strip().astype(bool)]

#
dfCleaned

,appID,reviewerName,reviewText,reviewerRating,reviewDate,textAnalytics
0,3,Eric Hansen,love well worth money full version came ad blo...,1.0,2017/07/07 00:00:00,NaN
1,3,Jacob N.,awful bug allow use space bar want type search...,0.4,2017/08/29 00:00:00,NaN
2,3,Higgins Family,would stars except bugs example incognito tab ...,0.8,2017/10/02 00:00:00,NaN
3,3,Rajko Dikmann,worked perfect weeks ago browsing experience s...,0.4,2017/09/28 00:00:00,NaN
4,3,Sergei Garcia,hands best browser play store even flagship de...,1.0,2017/07/09 00:00:00,NaN
...,...,...,...,...,...,...
111138,343,heera d,good less power consumer full review,0.8,1-September-2013,NaN
111139,343,Maulik Upadhyay,xperia pro nice change would nice full review,0.8,21-August-2013,NaN
111140,343,Lezlie Coleman,willl full review,0.6,24-September-2013,NaN
111141,343,Taher Bhai,great wonderful app samsung galaxy full review,1.0,14-August-2013,NaN


In [7]:
df['reviewText'][0]

type(df)


pandas.core.frame.DataFrame

In [6]:
dfCleaned[['reviewText']]

,reviewText
0,love well worth money full version came ad blo...
1,awful bug allow use space bar want type search...
2,would stars except bugs example incognito tab ...
3,worked perfect weeks ago browsing experience s...
4,hands best browser play store even flagship de...
...,...
111138,good less power consumer full review
111139,xperia pro nice change would nice full review
111140,willl full review
111141,great wonderful app samsung galaxy full review


## Vectorize words
### This is based on 
https://towardsdatascience.com/sentiment-analysis-a-how-to-guide-with-movie-reviews-9ae335e6bcb2
With actual logistic regression:
https://towardsdatascience.com/sentiment-classification-with-logistic-regression-analyzing-yelp-reviews-3981678c3b44


In [7]:
from sklearn.model_selection import train_test_split

dfReviews = dfCleaned['reviewText']
dfTrain, dfTest = train_test_split(dfReviews, test_size=.1)

In [8]:
dfTrain.head(10)

17960     unable register matter method used drops star ...
18764                                           full review
96536     thanks much dev mediatek support love app upda...
63810     nice dark background added thanks yet listenin...
100175    unlike everyone else stuck great game stars fu...
74331     amazing geocaching app feel like way better of...
828                         best app ever found full review
99534     fine get level beat literally tried times full...
2986                                       love full review
87686                                  ok great full review
Name: reviewText, dtype: object

In [9]:
# import statements
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Initialize a bag of words
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 1000) 

# Fit transform the data 
train_feat = vectorizer.fit_transform(dfTrain).toarray()
test_feat = vectorizer.transform(dfTest).toarray()


# TFIDF train set
tfidf_transformer = TfidfTransformer().fit(train_feat)
train_tfidf = tfidf_transformer.transform(train_feat)
 
# apply tfidf to test set
test_tfidf = tfidf_transformer.transform(test_feat)


In [42]:
dfTest
rv1 = pd.Series("bad terrible")
rv1_tf = tfidf_transformer.transform(vectorizer.transform(rv1))
# individual_x=cx_t.todense()
# clf.predict_proba(individual_x)

In [43]:
rv1_tf
cx_t = scipy.sparse.coo_matrix(rv1_tf)


print(rv1)
for i,j,v in zip(cx_t.row, cx_t.col, cx_t.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

0    bad terrible
dtype: object
(0, 866), terrible = 0.7920390356640833
(0, 65), bad = 0.6104704464462705


In [44]:
individual_x=cx_t.todense()
clf.predict_proba(individual_x)

array([[3.58405136e-04, 8.75000494e-01, 7.09665456e-02, 4.50875115e-02,
        8.58704402e-03]])

In [13]:
#train data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
#determined feature names
featureNamesList=vectorizer.get_feature_names()

In [14]:
# look at data for training
type(dfTrain)
stopcounter = 0
for index, value in dfTrain.items():
    print(f"Index : {index}, Value : {value}")
    stopcounter=stopcounter+1
    if stopcounter > 10:
        break

Index : 17960, Value : unable register matter method used drops star unable register matter method used full review
Index : 18764, Value : full review
Index : 96536, Value : thanks much dev mediatek support love app update working sony xa single sim phone android best regards lee full review
Index : 63810, Value : nice dark background added thanks yet listening user feedback still suggest making playing screen accessible main app tapping notification switch tracks intuitive full review
Index : 100175, Value : unlike everyone else stuck great game stars full review
Index : 74331, Value : amazing geocaching app feel like way better official one advertisiments best free full review
Index : 828, Value : best app ever found full review
Index : 99534, Value : fine get level beat literally tried times full review
Index : 2986, Value : love full review
Index : 87686, Value : ok great full review
Index : 4852, Value : works great full review


In [15]:
type(train_feat)
#len(train_feat)

numpy.ndarray

In [16]:
# Dump some vectorized words and probablities
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(dfTrain)

cx = scipy.sparse.coo_matrix(train_feat)
cx2 = scipy.sparse.coo_matrix(train_tfidf)


print(dfTrain.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))
    
print("\r\n\r\nThe weighted results\r\n")
for i,j,v in zip(cx2.row, cx2.col, cx2.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))

unable register matter method used drops star unable register matter method used full review
(0, 326), full = 1
(0, 523), matter = 2
(0, 725), review = 1
(0, 816), star = 1
(0, 912), unable = 2
(0, 931), used = 2


The weighted results

(0, 931), used = 0.4077722263468781
(0, 912), unable = 0.5814796439886868
(0, 816), star = 0.25343210340442224
(0, 725), review = 0.044723106111382215
(0, 523), matter = 0.6537393146892811
(0, 326), full = 0.044723106111382215


In [17]:
# look at some more data
import scipy.sparse

featureNamesList=vectorizer.get_feature_names()
type(test_tfidf)

cx = scipy.sparse.coo_matrix(test_tfidf)

print(dfTest.iloc[0])
for i,j,v in zip(cx.row, cx.col, cx.data):
    if i!=0:
        break
    print("(%d, %d), %s = %s" % (i,j,featureNamesList[j], v))


works great htc magic full review
(0, 986), works = 0.4565436659616101
(0, 725), review = 0.1315889458077755
(0, 388), htc = 0.7923073777774721
(0, 359), great = 0.3594239820492182
(0, 326), full = 0.1315889458077755


### Modelling part
1. Leverage the raw vector count and the tf-idf weighted version

In [18]:
# # Just looking at some data
# train_tfidf
# dfTrain
# print(df.iloc[105146])
# dfTrain
# df
# train_tfidf.todense()
# dfTrain.to_frame()

In [19]:
dfTest.head(10)
#type(dfTest)

70997                     works great htc magic full review
56590                                   awesome full review
79700     deactivate finger print scanner work active ev...
65876                            never got work full review
1458                                   nice app full review
93515     awesome best app android works perfectly htc o...
110748    perfect battery app keeper tried pretty much e...
25745     xp tracker count construction runecrafting hp ...
103015    works great android used app cyanogenmod andro...
46161     google integration note self google voice comm...
Name: reviewText, dtype: object

In [20]:
# Get trained Y data and test Y data
trainYdata = pd.merge(dfTrain.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)
testYdata = pd.merge(dfTest.to_frame(), dfCleaned[['reviewerRating']], left_index=True, right_index=True)

In [21]:
# Get trained X data and test X data
trainXdata = train_tfidf.todense()
testXdata = test_tfidf.todense()


In [22]:
print(dfCleaned[20520:20521])
print(df1[20624:20625])
print(testYdata[0:1])
print(testXdata[0:1])

       appID reviewerName                                         reviewText  \
20624   1410    Edgar Lim  nice work purchased unlocked features support ...   

       reviewerRating           reviewDate  textAnalytics  
20624             1.0  2016/08/14 00:00:00            NaN  
       appID reviewerName                                         reviewText  \
20624   1410    Edgar Lim  nice work purchased unlocked features support ...   

       reviewerRating           reviewDate  textAnalytics  
20624             1.0  2016/08/14 00:00:00            NaN  
                              reviewText  reviewerRating
70997  works great htc magic full review             1.0
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         

In [23]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [24]:
# Train data - map the y to ints of scales of reviews 0,1,2,3,4 reviews
y = trainYdata[['reviewerRating']]
y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
#y_int = trainYdata['reviewerRating'].apply(lambda x: 0 if x <.5 else 1)

X = trainXdata

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X, y_int)

# Check trained accuracy
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


0.820349585646472

In [25]:
clf.predict_proba(X[:2, :])

array([[0.00181229, 0.62981955, 0.09639813, 0.13978012, 0.13218991],
       [0.00087273, 0.10516011, 0.03123404, 0.05324641, 0.8094867 ]])

In [26]:
X[:2, :]

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

### Check test accuracy

In [27]:
y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.2 else (1 if x<.4 else (2 if x<.6 else (3 if x<.8 else 4))))
#y_int_test = testYdata['reviewerRating'].apply(lambda x: 0 if x<.5 else 1)

In [28]:
clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

0.8165278783490224

In [48]:
testXdata

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 10, n_jobs=-1)
# Train the model on training data
clt = rf.fit(X, y_int);

In [50]:
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y_int)


clf.predict(testXdata)
clf.predict_proba(testXdata)
clf.score(testXdata, y_int_test)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)